In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from bs4 import BeautifulSoup
import requests

In [5]:

# baixando os dados da petr4
data = yf.download("PETR4.SA", start="2024-12-01", end="2025-05-31") # intervalo de alguns meses
returns = np.log(data["Close"] / data["Close"].shift(1)) # log dos retornos
volDaily = returns.std() # desv pad dos retornos 
volAnnual = volDaily * np.sqrt(252) # anual = diario * sqrt(dias uteis)... desv. pad med. so 

print(f"Volatilidade histórica anual: {volAnnual.iloc[0]:.2%}")


[*********************100%***********************]  1 of 1 completed

Volatilidade histórica anual: 24.81%


In [ ]:
'''
------------------------------------------------

K --> strike --> scraping de https://opcoes.net.br/opcoes/bovespa/PETR4

------------------------------------------------

S0 --> google finance (media temporal, talvez) (feito; yfinance)

------------------------------------------------

sigma --> yfinance 

obs.: seria melhor usar volatilidade implicita ao inves de historica 

------------------------------------------------

r --> taxa selic? talvez... (feito)


'''

In [34]:
data.loc["2024-12-02"].iloc[0]

np.float64(35.40686798095703)

In [45]:
close = np.asarray([data.loc[date].iloc[0] for date in data.index])
open = np.asarray([data.loc[date].iloc[3] for date in data.index])

time = len(close)
avgClose = np.sum(close)/time
avgOpen = np.sum(open)/time

S0 = np.mean([avgClose, avgOpen])

In [210]:
import selenium
from selenium.webdriver.common.by import By

driver = selenium.webdriver.Chrome()

driver.get("https://www.bcb.gov.br/controleinflacao/historicotaxasjuros")
element = driver.find_element(by=By.CLASS_NAME, value = "table")

In [217]:
splitting = [i.split("-") for i in element.text.split("\n")[7:]]

In [212]:
import re

def processLine(linha):
    # Processa a primeira parte (antes da vírgula)
    parte1 = linha[0].strip()
    # Extrai o índice (ex: "269ª")
    indice = re.match(r'^\d+ª', parte1).group()
    # Extrai as duas primeiras datas (ex: "19/03/2025" e "20/03/2025")
    datas_parte1 = re.findall(r'\d{2}/\d{2}/\d{4}', parte1)
    data1, data2 = datas_parte1[0], datas_parte1[1] if len(datas_parte1) > 1 else 'n/a'
    
    # Processa a segunda parte (depois da vírgula)
    parte2 = linha[1].strip()
    # Extrai a terceira data (ex: "07/05/2025")
    data3 = re.search(r'\d{2}/\d{2}/\d{4}', parte2).group()
    # Extrai os números (ex: "14,25", "1,69", "14,15")
    numeros = re.findall(r'\d+,\d{2}', parte2)
    num1 = numeros[0] if len(numeros) > 0 else 'n/a'
    num2 = numeros[1] if len(numeros) > 1 else 'n/a'
    num3 = numeros[2] if len(numeros) > 2 else 'n/a'
    
    return [indice, data1, data2, data3, num1, 'n/a', num2, num3]

In [218]:
splitting = [processLine(line) for line in splitting[0:50]]

In [219]:
splitting = np.strings.replace(splitting, ",", ".")

In [220]:
df = pd.DataFrame(splitting)
df[[4, 6, 7]] = df[[4, 6, 7]].replace(',', '.', regex=True).astype(np.float32)
splitting = df.values.tolist()

In [227]:
foo = []
for line in splitting[0:3]:
    foo.append(line[-1])

In [229]:
r = np.mean(foo)